# Text Clustering

Los himnos nacionales son piezas musicales que representan la identidad y el orgullo de una nación, con letras que reflejan su historia, cultura y valores. Estos himnos varían considerablemente en estilo y contenido, desde melodías solemnes hasta composiciones enérgicas y vibrantes.

Algunos himnos tienen letras más militaristas, como el de Francia, que evocan un sentido de lucha y defensa. Otros himnos son más patrióticos y destacan la rica naturaleza y belleza del país, como el de Brasil. 

Esto lleva a pensar que las letras de los himnos podrían contener patrones que no son inmediatamente evidentes, y que  revelen similitudes y diferencias en las formas en que los países se representan a sí mismos a través de sus himnos, así como elementos de la cultura, historia y política de cada país.

## Importing Libraries:

In [ ]:
# Data Structures
import numpy  as np
import pandas as pd
#import geopandas as gpd
#import json

# Corpus Processing
import re
import nltk.corpus
from unidecode                        import unidecode
from nltk.tokenize                    import word_tokenize
from nltk                             import SnowballStemmer
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.preprocessing            import normalize

# K-Means
from sklearn import cluster

# Visualization and Analysis
import matplotlib.pyplot  as plt
import matplotlib.cm      as cm
import seaborn            as sns
from sklearn.metrics                  import silhouette_samples, silhouette_score
from wordcloud                        import WordCloud

# Map Viz
import folium
#import branca.colormap as cm
from branca.element import Figure

## Corpus Loading:

Usaremos pandas para leer el archivo CSV que contiene el himno nacional de cada país y su correspondiente código de país. Los himnos fueron extraídos de Wikipedia y muchos de ellos contienen palabras que utilizan caracteres no UTF-8 (generalmente nombres de lugares, etc.), por lo que leeremos el archivo con la codificación _latin1_.

Luego extraeremos la columna __Anthem__ en una lista de textos para nuestro corpus.

In [ ]:
data = pd.read_csv('datasets/anthems.csv', encoding='utf-8')
data.columns = map(str.lower, data.columns)

continents = ['Europe', 'South_America', 'North_America']
data = data.loc[data['continent'].isin(continents)]
data.head(6)

In [ ]:
corpus = data['anthem'].tolist()
corpus[18][0:447]

## Corpus Processing

### 1. Stop Words and Stemming

Realizaremos una rutina de ingeniería de datos con nuestro conjunto de datos de himnos para posteriormente crear un buen modelo. 

Para ello, eliminaremos todas las palabras que no contribuyan al significado semántico del texto (palabras que no están dentro del alfabeto inglés) y mantendremos todas las palabras restantes en el formato más simple posible, de modo que podamos aplicar una función que asigne pesos a cada palabra sin generar ningún sesgo o valores atípicos. Para ello, existen muchas técnicas para limpiar nuestro corpus, entre ellas, eliminaremos las palabras más comunes (stop words) y aplicaremos stemming, una técnica que reduce una palabra a su raíz.

Los métodos que aplican stemming y eliminación de stop words se enumeran a continuación. También definiremos un método que elimine cualquier palabra con menos de 2 letras o más de 21 letras para limpiar aún más nuestro corpus.

In [ ]:
# removes a list of words (ie. stopwords) from a tokenized list.
def removeWords(listOfTokens, listOfWords):
    return [token for token in listOfTokens if token not in listOfWords]

# applies stemming to a list of tokenized words
def applyStemming(listOfTokens, stemmer):
    return [stemmer.stem(token) for token in listOfTokens]

# removes any words composed of less than 2 or more than 21 letters
def twoLetters(listOfTokens):
    twoLetterWord = []
    for token in listOfTokens:
        if len(token) <= 2 or len(token) >= 21:
            twoLetterWord.append(token)
    return twoLetterWord

### 2. The main corpus processing function.

En una sección anterior, al explorar nuestro conjunto de datos, notamos algunas palabras que contenían caracteres extraños que deberían ser eliminados. Utilizando RegEx, nuestra función principal de procesamiento eliminará símbolos ASCII desconocidos, caracteres especiales, números, correos electrónicos, URLs, etc.

También utiliza las funciones auxiliares definidas anteriormente.

In [ ]:
def processCorpus(corpus, language):   
    stopwords = nltk.corpus.stopwords.words(language)
    param_stemmer = SnowballStemmer(language)
    countries_list = [line.rstrip('\n') for line in open('lists/countries.txt')] # Load .txt file line by line
    nationalities_list = [line.rstrip('\n') for line in open('lists/nationalities.txt')] # Load .txt file line by line
    other_words = [line.rstrip('\n') for line in open('lists/stopwords_scrapmaker.txt')] # Load .txt file line by line
    
    for document in corpus:
        index = corpus.index(document)
        corpus[index] = corpus[index].replace(u'\ufffd', '8')   # Replaces the ASCII '�' symbol with '8'
        corpus[index] = corpus[index].replace(',', '')          # Removes commas
        corpus[index] = corpus[index].rstrip('\n')              # Removes line breaks
        corpus[index] = corpus[index].casefold()                # Makes all letters lowercase
        
        corpus[index] = re.sub('\W_',' ', corpus[index])        # removes specials characters and leaves only words
        corpus[index] = re.sub("\S*\d\S*"," ", corpus[index])   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
        corpus[index] = re.sub("\S*@\S*\s?"," ", corpus[index]) # removes emails and mentions (words with @)
        corpus[index] = re.sub(r'http\S+', '', corpus[index])   # removes URLs with http
        corpus[index] = re.sub(r'www\S+', '', corpus[index])    # removes URLs with www

        listOfTokens = word_tokenize(corpus[index])
        twoLetterWord = twoLetters(listOfTokens)

        listOfTokens = removeWords(listOfTokens, stopwords)
        listOfTokens = removeWords(listOfTokens, twoLetterWord)
        listOfTokens = removeWords(listOfTokens, countries_list)
        listOfTokens = removeWords(listOfTokens, nationalities_list)
        listOfTokens = removeWords(listOfTokens, other_words)
        
        listOfTokens = applyStemming(listOfTokens, param_stemmer)
        listOfTokens = removeWords(listOfTokens, other_words)

        corpus[index]   = " ".join(listOfTokens)
        corpus[index] = unidecode(corpus[index])

    return corpus

In [ ]:
language = 'english'
corpus = processCorpus(corpus, language)
corpus[18][0:460]

### Statistical Weighting of Words

Ahora aplicaremos la función TF-IDF

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
tf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names_out())

final_df = tf_idf

print("{} rows".format(final_df.shape[0]))
final_df.T.nlargest(5, 0)

In [ ]:
# first 5 words with highest weight on document 0:
final_df.T.nlargest(5, 0)

#### Es momento de descubir los patrones presentes en los datos!!

In [ ]:
# TODO

## Cluster Analysis

Now we can take a deeper look at each cluster. 

In [ ]:
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices for each cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = vectorizer.get_feature_names_out()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs

def plotWords(dfs, n_feats):
    plt.figure(figsize=(8, 4))
    for i in range(0, len(dfs)):
        plt.title(("Most Common Words in Cluster {}".format(i)), fontsize=10, fontweight='bold')
        sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[i][:n_feats])
        plt.show()

#### Map of Words

Now that we can look at the graphs above and see the best scored words in each cluster, it's also interesting to make it prettier by making a map of words of each cluster!

In [ ]:
# Transforms a centroids dataframe into a dictionary to be used on a WordCloud.
def centroidsDict(centroids, index):
    a = centroids.T[index].sort_values(ascending = False).reset_index().values
    centroid_dict = dict()

    for i in range(0, len(a)):
        centroid_dict.update( {a[i,0] : a[i,1]} )

    return centroid_dict

def generateWordClouds(centroids):
    wordcloud = WordCloud(max_font_size=100, background_color = 'white')
    for i in range(0, len(centroids)):
        centroid_dict = centroidsDict(centroids, i)        
        wordcloud.generate_from_frequencies(centroid_dict)

        plt.figure()
        plt.title('Cluster {}'.format(i))
        plt.imshow(wordcloud)
        plt.axis("off")
        plt.show()

In [ ]:
centroids = pd.DataFrame(kmeans.cluster_centers_)
centroids.columns = final_df.columns
generateWordClouds(centroids)

### Preparing our final groups for visualization

Now that we're satisfied with our clustering we should assign which country belongs to which group.

In [ ]:
# Assigning the cluster labels to each country
# Create a dataframe named data_to_plot with a 'label' columns indicating the group number

### Visualization the Clustered Countries in a Map

Now that we have our final grouping it would be really cool to visualize it in a interactive map. To do this we'll use the awesome Folium library to see our interactive map!

We'll load a geojson file of polygons and country codes with geopandas and merge it with the labelled dataframe from the cell above.

In [ ]:
# Map Viz
import json
import geopandas as gpd

# Loading countries polygons
geo_path = 'datasets/world-countries.json'
country_geo = json.load(open(geo_path))
gpf = gpd.read_file(geo_path)

# Merging on the alpha-3 country codes
merge = pd.merge(gpf, data, left_on='id', right_on='alpha-3')
data_to_plot = merge[["id", "name", "label", "geometry"]]

data_to_plot.head(3)

Now we'll create a color_step for each group

In [ ]:
import branca.colormap as cm

# Creating a discrete color map
values = data_to_plot[['label']].to_numpy()
color_step = cm.StepColormap(['r', 'y','g','b', 'm'], vmin=values.min(), vmax=values.max(), caption='step')

color_step

### Painting the Groups into a Choropleth Map

Now that we have all the information that we want to plot into a Dataframe, we'll create a function that makes a Choropleth Map to be displayed on a folium map.

In [ ]:
import folium
from branca.element import Figure

def make_geojson_choropleth(display, data, colors):
    '''creates geojson choropleth map using a colormap, with tooltip for country names and groups'''
    group_dict = data.set_index('id')['label'] # Dictionary of Countries IDs and Clusters
    tooltip = folium.features.GeoJsonTooltip(["name", "label"], aliases=display, labels=True)
    return folium.GeoJson(data[["id", "name","label","geometry"]],
                          style_function = lambda feature: {
                               'fillColor': colors(group_dict[feature['properties']['id']]),
                               #'fillColor': test(feature),
                               'color':'black',
                               'weight':0.5
                               },
                          highlight_function = lambda x: {'weight':2, 'color':'black'},
                          smooth_factor=2.0,
                          tooltip = tooltip)

# Makes map appear inline on notebook
def display(m, width, height):
    """Takes a folium instance and embed HTML."""
    fig = Figure(width=width, height=height)
    fig.add_child(m)
    #return fig

In [ ]:
# Initializing our Folium Map
m = folium.Map(location=[43.5775, -10.106111], zoom_start=2.3, tiles='cartodbpositron')

# Making a choropleth map with geojson
geojson_choropleth = make_geojson_choropleth(["Country:", "Group:"], data_to_plot, color_step)
geojson_choropleth.add_to(m)

width, height = 1300, 675
display(m, width, height)
m